# Universidade de Brasília
## Programa de Pós-Graduação em Computação Aplicada
## Mestrato Profissional em Computação Aplicada
### Mineração de Dados
#### Professor: Marcelo Ladeira
#### Alunos: 
##### 22000xxxx - Alisson Melo Rios
##### 220005401 - Filipe Oliveira da Silva
##### 220005559 - Luís Eduardo Barreiro de Jesus


## 1. PREPARAÇÃO DO AMBIENTE DE PROCESSAMENTO

In [ ]:
# Preparação do Ambiente
## Seção de bibliotecas
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import datetime 
import folium
from folium.plugins import HeatMap, HeatMapWithTime, MarkerCluster


## 3. ESTATÍSTICA DESCRITIVA - Acidentes

### 3.1. Acidentes - Criar Dataframe

Criar o Dataframe com os dados de acidentes

In [ ]:
###
# Carga do arquivo gerado pelo processo de tratamento de dados
###
dftc = pd.read_csv("csv_tudo.csv", decimal=".", dtype={'anomes_nr': 'str'}, sep=",", encoding='Latin1') 
dftc = dftc.drop(columns=['Unnamed: 0'])

### 3.2. Quantitativo de Acidentes

#### 3.2.1 Acidentes x Classificação do Acidente


# ANTES DA CONCESSÃO

In [ ]:
dfa = dftc[(dftc['pos_concessao']==0)&((dftc['classificacao_acidente']!='SEM VÍTIMAS') & (dftc['classificacao_acidente']!='IGNORADO'))].groupby(['anomes_nr']).agg({'id':'count', 'ilesos':'mean', 'feridos_leves':'mean', 'feridos_graves':'mean', 'mortos':'mean'})
dfa

In [ ]:
dfa.describe()

In [ ]:
## ANTES DA CONCESSÃO - TODO O TRECHO CONCEDIDO
# Somente acidentes com vítimas**
def _selecao_dados(df, secao):
    
    if secao  == 0:
        v_msg = 'ANÁLISE DESCRITIVA DE TODO O TRECHO CONCEDIDO'
        # dados de antes da concessão
        dfa = df[(dftc['pos_concessao']==0)&((df['classificacao_acidente']!='SEM VÍTIMAS') & (df['classificacao_acidente']!='IGNORADO'))].groupby(['anomes_nr']).agg({'id':'count', 'ilesos':'mean', 'feridos_leves':'mean', 'feridos_graves':'mean', 'mortos':'mean'})
        # dados de depois da concessão
        dfd = df[(dftc['pos_concessao']==1)&((df['classificacao_acidente']!='SEM VÍTIMAS') & (df['classificacao_acidente']!='IGNORADO'))].groupby(['anomes_nr']).agg({'id':'count', 'ilesos':'mean', 'feridos_leves':'mean', 'feridos_graves':'mean', 'mortos':'mean'})
    else:
        v_msg = 'ANÁLISE DESCRITIVA - DA SEÇÃO : %s ' %  secao  
        # dados de antes da concessão
        dfa = df[(dftc['secao']==secao)&(dftc['pos_concessao']==0)&((df['classificacao_acidente']!='SEM VÍTIMAS') & (df['classificacao_acidente']!='IGNORADO'))].groupby(['anomes_nr']).agg({'id':'count', 'ilesos':'mean', 'feridos_leves':'mean', 'feridos_graves':'mean', 'mortos':'mean'})
        # dados de depois da concessão
        dfd = df[(dftc['secao']==secao)&(dftc['pos_concessao']==1)&((df['classificacao_acidente']!='SEM VÍTIMAS') & (df['classificacao_acidente']!='IGNORADO'))].groupby(['anomes_nr']).agg({'id':'count', 'ilesos':'mean', 'feridos_leves':'mean', 'feridos_graves':'mean', 'mortos':'mean'})

    print(v_msg)
        
    return dfa, dfd


def _prints_estatisticas(dfapre, dfapos):

    
    print('A média de mortes por acidentes pré concessão : %.4f' % dfapre.describe().mortos['mean'])
    print('A média de mortes por acidentes pos concessão : %.4f' % dfapos.describe().mortos['mean'])
    print('A diferença é : %.4f' % ((dfapos.describe().mortos['mean'].astype('float') / dfapre.describe().mortos['mean'].astype('float')) - 1))

    print('A média de feridos_graves por acidentes pré concessão : %.4f' % dfapre.describe().feridos_graves['mean'])
    print('A média de feridos_graves por acidentes pós concessão : %.4f' % dfapos.describe().feridos_graves['mean'])
    print('A diferença é : %.4f' % ((dfapos.describe().feridos_graves['mean'].astype('float') / dfapre.describe().feridos_graves['mean'].astype('float')) - 1))

    print('A média de feridos_leves por acidentes pré concessão : %.4f' % dfapre.describe().feridos_leves['mean'])
    print('A média de feridos_leves por acidentes pós concessão : %.4f' % dfapos.describe().feridos_leves['mean'])
    print('A diferença é : %.4f' % ((dfapos.describe().feridos_leves['mean'].astype('float') / dfapre.describe().feridos_leves['mean'].astype('float')) - 1))

    print('A média de acidentes por acidentes pré concessão: %.4f' % dfapre.describe().ilesos['mean'])
    print('A média de acidentes por acidentes pós concessão: %.4f' % dfapos.describe().ilesos['mean'])
    print('A diferença é : %.4f' % ((dfapos.describe().ilesos['mean'].astype('float') / dfapre.describe().ilesos['mean'].astype('float')) - 1))

    print('A média de acidentes por mês pré concessão : %.4f' % dfapre.describe().id['mean'])
    print('A média de acidentes por mês pos concessão : %.4f' % dfapos.describe().id['mean'])
    print('A diferença é : %.4f' % ((dfapos.describe().id['mean'].astype('float') / dfapre.describe().id['mean'].astype('float')) - 1))
    

In [ ]:
dfa, dfd = _selecao_dados(dftc, 0)
_prints_estatisticas(dfa, dfd)

In [ ]:
dfa, dfd = _selecao_dados(dftc, 2)
_prints_estatisticas(dfa, dfd)

In [ ]:
dfa, dfd = _selecao_dados(dftc, 3)
_prints_estatisticas(dfa, dfd)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def _get_acidentes_trechos_duplicados(dfacid, secao, pos_concessao):
   
    if secao == 5: # código para toda a série histórica | não considera pré ou pós concessão
        #dfa = dfacid.groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'mortos':'sum'})
        dfa = dfacid[(dfacid['trecho_duplicado']==1)].groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'mean', 'feridos_leves':'mean', 'feridos_graves':'mean', 'mortos':'mean'})
    else: 
        dfa = dfacid[(dfacid['trecho_duplicado']==1)&(dfacid['secao']==secao)&(dfacid['pos_concessao']==pos_concessao)&((dfacid['classificacao_acidente']!='SEM VÍTIMAS') & (dfacid['classificacao_acidente']!='IGNORADO'))&(dfacid['trecho_duplicado']==1)].groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'mortos':'sum'})
    #dfa = dfacid.groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'mortos':'sum'})
    dfa = pd.pivot_table(dfa, values='id', index=['anomes_nr'],
                         columns=['classificacao_acidente'], aggfunc=np.sum)
    #dfa.reset_index(inplace=True)
    dfa.reset_index(inplace=True)
    dfa.fillna(0, inplace=True)
    return dfa



def _get_acidentes(dfacid, secao, pos_concessao):
    # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # Quantitativo de acidentes por classificação de acidente
    # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    
    if secao == 5: # código para toda a série histórica | não considera pré ou pós concessão
        #dfa = dfacid.groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'mortos':'sum'})
        dfa = dfacid.groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'mean', 'feridos_leves':'mean', 'feridos_graves':'mean', 'mortos':'mean'})
    else: 
        dfa = dfacid[(dfacid['secao']==secao)&(dfacid['pos_concessao']==pos_concessao)&((dfacid['classificacao_acidente']!='SEM VÍTIMAS') & (dfacid['classificacao_acidente']!='IGNORADO'))].groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'mortos':'sum'})
    #dfa = dfacid.groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'mortos':'sum'})
    dfa = pd.pivot_table(dfa, values='id', index=['anomes_nr'],
                         columns=['classificacao_acidente'], aggfunc=np.sum)
    #dfa.reset_index(inplace=True)
    dfa.reset_index(inplace=True)
    dfa.fillna(0, inplace=True)
    return dfa


def _plot_line_graf(dfa, secao, pos_concessao):
    # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # Quantitativo de acidentes por classificação de acidente
    # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    if pos_concessao == 0:
        tit = 'Pré concessão'
    elif pos_concessao == 1:
        tit = 'Pós concessão'        
    else:
        tit = 'Toda a série'
    
    titulo = f'Acidentes na Seção {secao} - {tit}'
    
    figura = px.line(title = titulo, width=1100, height=400)
    for i in dfa.columns[1:]:
        figura.add_scatter(x = dfa["anomes_nr"] ,y = dfa[i], name = i)
        #figura.add_scatter(x = table["anomes_nr"] ,y = table[i], name = i)
    figura.show()
    
    dfa.rename(columns={'COM VÍTIMAS FATAIS':'COM_VÍTIMAS_FATAIS', 'COM VÍTIMAS FERIDAS':'COM_VÍTIMAS_FERIDAS'}, inplace=True)
               
    return dfa

def _plot_heatmap(dfacid, secao, pos_concessao, tipo_mapa):
   
    ### 
    # Filtro para plot
    dfa = dfacid[(dfacid['secao']==secao)&(dfacid['pos_concessao']==pos_concessao)&((dfacid['classificacao_acidente']!='SEM VÍTIMAS') & (dfacid['classificacao_acidente']!='IGNORADO'))]
    ###
    # criar lista de ano_mes únicos
    lista_index = dfa.ano_mes.unique()

    #print(lista_index)

    lista_idx = []
    for i in lista_index:
        lista_idx.append(i)

    #print(lista_idx)
    # -------------------------------------------

    # criar a weight list
    weight_list = []
    dfa['conta'] = 1
    for x in dfa['ano_mes'].sort_values().unique():
        weight_list.append(dfa.loc[dfa['ano_mes'] == x, ["latitude", "longitude", "conta"]].groupby(["latitude", "longitude"]).sum().reset_index().values.tolist()) 

    # -------------------------------------------

    # criar o mapa base
    my_map = folium.Map(
                    location=(
                                (dfa.latitude.mean()),
                                (dfa.longitude.mean())
                            ),
                    zoom_start=6
                )

    #folium.LayerControl().add_to(my_map)
    folium.TileLayer('stamentoner').add_to(my_map)

    # criar o mapa de calor de acidentes
    HeatMapWithTime(weight_list, radius=10, index=lista_idx,
                    gradient={0.1: 'blue', 0.25: 'yellow', 0.75:'orange', 1:'red'},
                    auto_play=True, min_opacity=0.5, max_opacity=1, use_local_extrema=True).add_to(my_map)

    return my_map


def _plot_heatmap2(dfacid, secao, pos_concessao, tipo_mapa):
   
    ### 
    # Filtro para plot
    if secao == 5:  # todo o trecho
        dfa = dfacid[(dfacid['classificacao_acidente']=='COM VÍTIMAS FATAIS')&((dfacid['km']!=511)&(dfacid['km']!=515))].groupby(["latitude", "longitude"]).agg({'id':'count'})
        
    else:
        dfa = dfacid[(dfacid['secao']==secao)&(dfacid['pos_concessao']==pos_concessao)&(dfacid['classificacao_acidente']=='COM VÍTIMAS FATAIS')&((dfacid['km']!=511)&(dfacid['km']!=515))].groupby(["latitude", "longitude"]).agg({'id':'count'})
    
    dfa.reset_index(inplace=True)
    dfa = dfa.dropna()
        
    # criar o mapa base
    my_map2 = folium.Map(
                    location=(
                                (dfa.latitude.mean()),
                                (dfa.longitude.mean())
                            ),
                    zoom_start=6
                )
    folium.TileLayer('stamentoner').add_to(my_map2)
    HeatMap(dfa).add_to(my_map2)
    return my_map2


def _plot_cluster_map(dfacid, secao, pos_concessao, tipo_mapa):

    from folium.plugins import MarkerCluster

    if secao == 5:
        dfa = dfacid[((dfacid['classificacao_acidente']=='COM VÍTIMAS FATAIS')&((dfacid['km']!=511)&(dfacid['km']!=515)))]
    else:
        dfa = dfacid[(dfacid['secao']==secao)&(dfacid['pos_concessao']==pos_concessao)&((dfacid['classificacao_acidente']=='COM VÍTIMAS FATAIS')&((dfacid['km']!=511)&(dfacid['km']!=515)))]
    
    dfa = dfa.dropna()
    
    m = folium.Map(
                    location=(
                                (dfa.latitude.mean()),
                                (dfa.longitude.mean())
                            ),
                    zoom_start=6
                )
    
    marker_cluster = MarkerCluster().add_to(m)

    for index, row in dfa.iterrows():
            #print(row['c1'], row['c2']):
        
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup= f"Acidente fatal: Ano_mes=%s, id=%s, latitude=%s, longitude=%s, UF=%s, KM=%s" % (row['ano_mes'],row['id'],row['latitude'], row['longitude'], row['uf'], row['km']),
            icon=folium.Icon(color="green", icon="ok-sign"),
            ).add_to(marker_cluster)
        '''
        folium.Marker(
            location=[44.67, -73.94],
            popup="Add popup text here.",
            icon=folium.Icon(color="red", icon="remove-sign"),
        ).add_to(marker_cluster)

        folium.Marker(
            location=[44.67, -71.94],
            popup="Add popup text here.",
            icon=None,
        ).add_to(marker_cluster)
        '''
    return m

In [ ]:
# Seções - 1 (DF até Cristalina) | 2 (Cristalina até Curvelo) | 3 (Curvelo até Juiz de Fora) | 5 TODA A SÉRIE HISTÓRICA
# Pos Concessão - 0 (antes da concessão -> até junho de 2014) | 1 (a partir de julho de 2014)

secao = 5
pos_concessao = 0
dfa = _plot_line_graf(_get_acidentes(dftc,secao,pos_concessao), secao,pos_concessao)

print('A média de mensal de acidente com Vítimas Fatais : %.4f' % dfa.describe().COM_VÍTIMAS_FATAIS['mean'])
print('A média de mensal de acidente com Vítimas Feridas : %.4f' % dfa.describe().COM_VÍTIMAS_FERIDAS['mean'])

#dfa.describe() 


In [ ]:
# Seções - 1 (DF até Cristalina) | 2 (Cristalina até Curvelo) | 3 (Curvelo até Juiz de Fora) | 5 TODA A SÉRIE HISTÓRICA
# Pos Concessão - 0 (antes da concessão -> até junho de 2014) | 1 (a partir de julho de 2014)

secao = 5
pos_concessao = 1
dfa = _plot_line_graf(_get_acidentes(dftc,secao,pos_concessao), secao,pos_concessao)
print('A média de mensal de acidente com Vítimas Fatais : %.4f' % dfa.describe().COM_VÍTIMAS_FATAIS['mean'])
print('A média de mensal de acidente com Vítimas Feridas : %.4f' % dfa.describe().COM_VÍTIMAS_FERIDAS['mean'])



# ACHADO

Como houve uma grande redução da quantidade dos registros de acidentes classificados como "Sem Vítimas" a partir de 2014, este estudo vai desconsiderar os registros. Portanto, o estudo vai considerar somente os registros de acidentes classificados como "Com Vítimas Feridas" e "Com Vítimas Fatais".



##### 3.2.1.1. Acidentes na Seção 1 - Antes da Concessão


In [ ]:
# Seções - 1 (DF até Cristalina) | 2 (Cristalina até Curvelo) | 3 (Curvelo até Juiz de Fora)
# Pos Concessão - 0 (antes da concessão -> até junho de 2014) | 1 (a partir de julho de 2014)

secao = 1
pos_concessao = 0
dfa = _plot_line_graf(_get_acidentes(dftc,secao,pos_concessao), secao,pos_concessao)
print('A média de mensal de acidente com Vítimas Fatais : %.4f' % dfa.describe().COM_VÍTIMAS_FATAIS['mean'])
print('A média de mensal de acidente com Vítimas Feridas : %.4f' % dfa.describe().COM_VÍTIMAS_FERIDAS['mean'])



In [ ]:
# Seções - 1 (DF até Cristalina) | 2 (Cristalina até Curvelo) | 3 (Curvelo até Juiz de Fora)
# Pos Concessão - 0 (antes da concessão -> até junho de 2014) | 1 (a partir de julho de 2014)

secao = 1
pos_concessao = 1
dfa = _plot_line_graf(_get_acidentes(dftc,secao,pos_concessao), secao,pos_concessao)
print('A média de mensal de acidente com Vítimas Fatais : %.4f' % dfa.describe().COM_VÍTIMAS_FATAIS['mean'])
print('A média de mensal de acidente com Vítimas Feridas : %.4f' % dfa.describe().COM_VÍTIMAS_FERIDAS['mean'])


In [ ]:
# Seções - 1 (DF até Cristalina) | 2 (Cristalina até Curvelo) | 3 (Curvelo até Juiz de Fora)
# Pos Concessão - 0 (antes da concessão -> até junho de 2014) | 1 (a partir de julho de 2014)

secao = 2
pos_concessao = 0
dfa = _plot_line_graf(_get_acidentes(dftc,secao,pos_concessao), secao,pos_concessao)
print('A média de mensal de acidente com Vítimas Fatais : %.4f' % dfa.describe().COM_VÍTIMAS_FATAIS['mean'])
print('A média de mensal de acidente com Vítimas Feridas : %.4f' % dfa.describe().COM_VÍTIMAS_FERIDAS['mean'])


##### 3.2.1.2. Acidentes na Seção 1 - Pós da Concessão

In [ ]:
# Seções - 1 (DF até Cristalina) | 2 (Cristalina até Curvelo) | 3 (Curvelo até Juiz de Fora)
# Pos Concessão - 0 (antes da concessão -> até junho de 2014) | 1 (a partir de julho de 2014)

secao = 2
pos_concessao = 1
dfa = _plot_line_graf(_get_acidentes(dftc,secao,pos_concessao), secao,pos_concessao)
print('A média de mensal de acidente com Vítimas Fatais : %.4f' % dfa.describe().COM_VÍTIMAS_FATAIS['mean'])
print('A média de mensal de acidente com Vítimas Feridas : %.4f' % dfa.describe().COM_VÍTIMAS_FERIDAS['mean'])


In [ ]:
# Seções - 1 (DF até Cristalina) | 2 (Cristalina até Curvelo) | 3 (Curvelo até Juiz de Fora)
# Pos Concessão - 0 (antes da concessão -> até junho de 2014) | 1 (a partir de julho de 2014)

secao = 3
pos_concessao = 0
dfa = _plot_line_graf(_get_acidentes(dftc,secao,pos_concessao), secao,pos_concessao)
print('A média de mensal de acidente com Vítimas Fatais : %.4f' % dfa.describe().COM_VÍTIMAS_FATAIS['mean'])
print('A média de mensal de acidente com Vítimas Feridas : %.4f' % dfa.describe().COM_VÍTIMAS_FERIDAS['mean'])


In [ ]:
# Seções - 1 (DF até Cristalina) | 2 (Cristalina até Curvelo) | 3 (Curvelo até Juiz de Fora)
# Pos Concessão - 0 (antes da concessão -> até junho de 2014) | 1 (a partir de julho de 2014)

secao = 3
pos_concessao = 1
dfa = _plot_line_graf(_get_acidentes(dftc,secao,pos_concessao), secao,pos_concessao)
print('A média de mensal de acidente com Vítimas Fatais : %.4f' % dfa.describe().COM_VÍTIMAS_FATAIS['mean'])
print('A média de mensal de acidente com Vítimas Feridas : %.4f' % dfa.describe().COM_VÍTIMAS_FERIDAS['mean'])


In [ ]:
dftc.isna().sum()

In [ ]:
#_get_acidentes_trechos_duplicados

# TRECHO DUPLICADO

# Seções - 1 (DF até Cristalina) | 2 (Cristalina até Curvelo) | 3 (Curvelo até Juiz de Fora)
# Pos Concessão - 0 (antes da concessão -> até junho de 2014) | 1 (a partir de julho de 2014)

secao = 5
pos_concessao = 1
dfa = _plot_line_graf(_get_acidentes_trechos_duplicados(dftc,secao,pos_concessao), secao, pos_concessao)
print('A média de mensal de acidente com Vítimas Fatais : %.4f' % dfa.describe().COM_VÍTIMAS_FATAIS['mean'])
print('A média de mensal de acidente com Vítimas Feridas : %.4f' % dfa.describe().COM_VÍTIMAS_FERIDAS['mean'])
#print('A média de mensal de acidente : %.4f' % dfa.describe().id['mean'])

dfa.describe()


# ACIDENTES - SHAP - CLASSIFICAÇÃO XGBOOST


In [ ]:
# importe do cross_val_score

import statsmodels.api as sm
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.model_selection import train_test_split, cross_val_score
import shap
import category_encoders as ce
import xgboost as xgb

def _get_data2classifier(df, secao, concessao, tdup):
    # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
    # Prepara o DataFrame 
    # ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

    if tdup == True:
        df = df[(df['trecho_duplicado']==1)&(df['pos_concessao']==concessao)&(df['classificacao_acidente']!='SEM VÍTIMAS')] #.groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'mortos':'sum'})
        print('Trecho duplicado')
    else:
        # Filtro no DataFrame para seção e concessão
        df = df[(df['secao']==secao)&(df['pos_concessao']==concessao)&(df['classificacao_acidente']!='SEM VÍTIMAS')] #.groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'mortos':'sum'})
        print('Secao = %s e Pos_concessao = %s' % (secao, concessao))
    # total
    #df = dftc[(dftc['classificacao_acidente']!='SEM VÍTIMAS')] #.groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'mortos':'sum'})

    # Target - Acidentes Graves (mortos ou feridos graves)
    #df['acidente_grave'] = df['acidente_grave'].map({'COM VÍTIMAS FERIDAS':0,'COM VÍTIMAS FATAIS':1,'IGNORADO':2, 'SEM VÍTIMAS':3})
    df['acidente_grave'] = 0
    df.loc[df['mortos'] > 0, 'acidente_grave'] = 1
    df.loc[df['feridos_graves'] > 0, 'acidente_grave'] = 1

    # target para acidentes com Vítimas graves e fatais = 1, e Vítimas leves = 0
    df.dropna(inplace=True)

    # map para o tipo de pista - nosso target
    #df['tipo_pista'] = df['tipo_pista'].map({'DUPLA':1,'MÚLTIPLA':1,'SIMPLES':0, '(NULL)':0})

    # Drop nas colunas que não serão usadas na análise
    df = df.drop(columns=[ 'id',
                           'data_inversa',
                           'br',
                           'latitude',                
                           'longitude',
                           'kmint',
                           'ano',
                           'ano_mes', 
                           'anomes_nr',
                           'km',
                           'hora',
                           'mes',
                           'municipio',
                           'uso_solo',
                           'secao',
                           'pos_concessao',
                           'minuto',
                           'ilesos',
                           'feridos_leves',
                           'feridos_graves',
                           'classificacao_acidente',
                           'mortos'])

    #df['anomes_nr'] = df['anomes_nr'].astype('int')

    #xx = df
    
    # Preparação dos dados 
    df = pd.get_dummies(df)

    # Filtro para pegar por tipo de acidente
    #df = df[(df['acidente_grave']==1)&(df['tipo_acidente_ATROPELAMENTO DE PESSOA']>=1)] #.groupby(['anomes_nr','classificacao_acidente']).agg({'id':'count', 'ilesos':'sum', 'feridos_leves':'sum', 'feridos_graves':'sum', 'mortos':'sum'})

    # Definição do X e y para o ML
    X = df.drop(['acidente_grave'], axis=1)
    y = df['acidente_grave']

    # Split para o modelo
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=13)
    # retornar os dados
    return X_train, X_test, y_train, y_test
    

def _get_xgb_classificador(X_train, y_train):
    classificador_xgb = xgb.XGBClassifier()
    print('Classificador sem parâmetros = %.4f' % (100 * cross_val_score(classificador_xgb, X_train, y_train).mean()))
    
    classificador_xgb_tunado = xgb.XGBClassifier(max_depth=2)
    print('Classificador com max_depth 2 = %.4f' % (100 * cross_val_score(classificador_xgb_tunado, X_train, y_train).mean()))
    
    classificador_xgb_dart = xgb.XGBClassifier(booster='dart')
    print('Classificador com booster "dart" = %.4f' % (100 * cross_val_score(classificador_xgb_dart, X_train, y_train).mean()))

    classificador_campeao = classificador_xgb_tunado
    
    return classificador_campeao

    
    

In [ ]:
# Formata os dados para o modelo
secao = 3
pos_concessao = 1
tdupl = False
dfx, X_train, X_test, y_train, y_test = _get_data2classifier(dftc, secao, pos_concessao, tdupl)

In [ ]:
dfx.info()

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
classificador_campeao = _get_xgb_classificador(X_train, y_train)

In [ ]:
model = classificador_campeao.fit(X_train, y_train)
model 

In [ ]:
y_pred_train_proba = classificador_campeao.predict_proba(X_train)
print('Probabilidade do modelo prever acidentes não graves = ',(100*y_pred_train_proba[3][0]).round(2),'%.')
print('Probabilidade do modelo prever acidentes graves = ',(100*y_pred_train_proba[3][1]).round(2),'%.')



In [ ]:
predict = classificador_campeao.predict(X_test)
acuracia_teste = 100* (predict == y_test).sum() / len(y_test)
print('No teste, o modelo acertou {:.2f}% das previsões na base de teste.'.format(acuracia_teste))

In [ ]:
xgb.plot_importance(classificador_campeao, max_num_features=15)

In [ ]:
# this takes a minute or two since we are explaining over 30 thousand samples in a model with over a thousand trees
explainer = shap.TreeExplainer(model=classificador_campeao)
shap_values = explainer.shap_values(X_train, y_train)

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="bar", plot_size=(15,8));

In [ ]:
shap.summary_plot(shap_values, X_train, plot_type="dot", plot_size=(15,8))

In [ ]:
#for name in X_train.columns:
#    shap.dependence_plot(name, shap_values, X_train, display_features=X_train)

# MAPAS

In [ ]:
## SEÇÃO 5 É TODO O TRECHO - TODO O PERÍODO - VÍTIMAS FATAIS SOMENTE
m2 = _plot_heatmap2(dftc, 5, pos_concessao, 0)
m2

In [ ]:
# plotar mapa de calor
m = _plot_heatmap(dftc, secao, pos_concessao, 0)
m

In [ ]:
m2 = _plot_cluster_map(dftc, 5, pos_concessao, 0)
m2

In [ ]:
# plotar mapa de calor
m = _plot_heatmap(dftc, secao, pos_concessao, 0)
m

In [ ]:
m2 = _plot_cluster_map(dftc, secao, pos_concessao, 0)
m2


In [ ]:
# plotar mapa de calor
m = _plot_heatmap(dftc, secao, pos_concessao, 0)
m

In [ ]:
m2 = _plot_cluster_map(dftc, secao, pos_concessao, 0)
m2

In [ ]:
# plotar mapa de calor
m = _plot_heatmap(dftc, secao, pos_concessao, 0)
m

In [ ]:
m2 = _plot_cluster_map(dftc, secao, pos_concessao, 0)
m2

In [ ]:
# plotar mapa de calor
m = _plot_heatmap(dftc, secao, pos_concessao, 0)
m

In [ ]:
m2 = _plot_cluster_map(dftc, secao, pos_concessao, 0)
m2

In [ ]:
# plotar mapa de calor
m = _plot_heatmap(dftc, secao, pos_concessao, 0)
m

In [ ]:
m2 = _plot_cluster_map(dftc, secao, pos_concessao, 0)
m2

# TRECHO DUPLICADO

In [ ]:
XX = _get_acidentes_trechos_duplicados(dftc, 5, 0)

In [ ]:
#XX.fillna(0, inplace=True)

In [ ]:
XX.info()